In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    import os
    import sys
    os.chdir('/content/drive/MyDrive/DL_Project_GD/')
    sys.path.append('/content/drive/MyDrive/DL_Project_GD/code/')

    #%% [code]
    ! pip install -qqq "torchvision" "torch>=1.6, <1.9" "torchmetrics>=0.3" "ipython[notebook]" "pytorch-lightning>=1.3" "torchtext"
    ! pip install -qqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

    ## Install dependencies
    ! pip install -qqq wget
    ! apt-get -qqq install sox libsndfile1 ffmpeg
    ! pip install -qqq unidecode
    ! pip install -qqq matplotlib>=3.3.2

    ## Install NeMo
    BRANCH = 'r1.6.1'
    ! python -m pip install -qqq git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

    ! mkdir configs
    ! wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

    datapath = '../dataset/raw'
    rootpath = './'
    print("Running with Colab setup.")
    tpu_cores = 8
    gpu_cores = None
except(ModuleNotFoundError):
    print('Initialising non-colab set-up...')
    tpu_cores=None
    gpu_cores=None
    datapath = '../dataset/raw/'
    rootpath = '../'

from VGGultramini import VGGultramini
from dataloader import VoxDataloader
import runner
import os
print("cwd: {}".format(os.getcwd()))
print("datapath: {}".format(datapath))
print('done.')

Initialising non-colab set-up...
cwd: /Users/jameswilkinson/Documents/python programs/DL_Project/code
datapath: /Users/jameswilkinson/Downloads/dev/aac/
done.


In [3]:
config = {
            # learning params
            'lr': 1e-3,
            'batch_size': 64,
            'early_stopping': True,
            'optimizer': 'Adam',
            'max_epochs': 20,
            'early_stopping': True,
            'patience': 3, # tolerance for early stopping
            'momentum': 0.,
            'lr_decay': 0.,

            # model
            'fftmethod': 'librosa.stft', # choose librosa.stft, signal.stft, or librosa.mfcc
            'pooling' : 'average', # 'average' or 'max'. will not have effect if not using VGGpool

            #, regularization
            'dropout': 0.0,
            'L2': 0.0,
            'batch_norm': True,
            'gradient_clipping': 0.0,
}

In [4]:
for method in ['librosa.stft', 'librosa.mel', 'librosa.mfcc','pydct.sdct']:
  config['fftmethod'] = method
  model = runner.run(config, datapath, VoxDataloader, VGGultramini, './VGGultramini/', tpu_cores=tpu_cores, gpu_cores=gpu_cores, phase_map_file='phase_map_40.csv')


LOGNAME: spctlibrosa.stft_optAdam_lr0.001_reg0.0_drop0.0_bnTrue_gc0.0_mom0.0_dec0.0_poolingaverage


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: ./VGGultramini/spctlibrosa.stft_optAdam_lr0.001_reg0.0_drop0.0_bnTrue_gc0.0_mom0.0_dec0.0_poolingaverage

   | Name       | Type             | Params
-------------------------------------------------
0  | loss       | CrossEntropyLoss | 0     
1  | activation | ReLU             | 0     
2  | conv1      | Conv2d           | 4.8 K 
3  | mpool1     | MaxPool2d        | 0     
4  | conv2      | Conv2d           | 307 K 
5  | mpool2     | MaxPool2d        | 0     
6  | fc3        | Linear           | 32    
7  | fc4        | Linear           | 18    
8  | fc5        | Linear           | 4.5 K 
9  | seq1       | Sequential       | 192   
10 | seq2       | Sequential       | 256   
11 | seq3       | Sequential       | 256   
-------------------------------------------------
317 K     Trainable params
0         Non-trainable params
317 K     Total params
1.270  

N classes: 35


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

KeyboardInterrupt: 